# Homework 3 - Which book would you recomend?

![alt text](img/SA_GR_SE_NL.png "")

##### **Authors:** Daniel Jiménez & Beatrice Nobille & Selim Yahiamessaoud
##### **Date:** 2020/12/06
##### **Subject:** Algorithmic Methods of Data Mining
##### **Data Science Master's programme**
##### **La Sapienza University of Rome**
_____

This file contains the solution to all the five Challenges (and its corresponding literals) required for the web scrapping and search engine creation for the Best Books ever list that you can find [HERE](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1)

As like in any data-wrangling case, it's relevant to keep clean and tidy the working environment. We achieve this by explaining step-by-step the process we followed to crawl, parse and store the attributes of the books. 

**Notes:** 
- We worked on a PC with an SSD of 50 GB of free space and a RAM of 16 GB.

Well, first thing first, we set the libraries that we will be using to perform all the web scraping and search engine creation. 

# 1. Data collection

First things first. We import the required libraries to solve the challenges.

In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.firefox import GeckoDriverManager
import time
import csv
from bs4 import BeautifulSoup
import requests
import pathlib
import math
import os
import pandas as pd
import numpy as np
import re
from langdetect import detect
import our_functions

## 1.1. Get the list of books

To get the list of the links for each book we followed the next steps:
1. We used `Selenium`, opening a browser (Firefox) to access to the main page where the list of the Best Books Ever
2. Then, we look for all the links that are on the page that contains '/book/show/' which have the URL we are looking for
3. As there are repeated links (appearing in links itself and in the photo of the book) we removed the duplicated values
4. After running in a loop the steps 2 and 3 for each of the 300 first pages, we stored them in the file **books_urls.txt** to be used later. We avoided saving two links of that were out of the list but inside the comments. Those books are *The War of the Worlds* and The *Shack*

Note: In some occasions, we used `time.sleep(n)` to force the PC to wait n seconds until the webpage has been completely loaded to avoid errors when looking for an object. 

In [63]:
start_time = time.time()
# Open driver (in this case Firefox - need to be installed in the PC)
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# Open in Firefox the desired url
href = 'https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1'
driver.get(href)

# Pause execution while the page loads the content
time.sleep(12)

# Iterate over each page to retrieve book's links
urls = []
for i in range(300):
    partial_time1 = time.time()
    # Find all elements that contain a link inside a table (td)
    elements = driver.find_elements_by_xpath("//a[@href]")
    
    for elem in elements:
        # Retrieve the link
        link = elem.get_attribute("href")
        # Check if the link corresponds to a book
        if '/book/show/' in link:
            # Append each link found
            urls.append(link)
    
    # Click on "next" link to advance in each webpage
    driver.find_element_by_partial_link_text("next").click()
    
    # Pause execution while the page loads the content
    time.sleep(5)
    print("Retrieving page", i + 1, "/ 300 took", time.time() - partial_time1, "seconds")
    
# Remove duplicated links (keeping the order of appearence of each book's url)
# final_urls = list(set(urls))
final_urls = []
for i in urls:
    if i not in final_urls:
        final_urls.append(i)

# Close Firefox browser
driver.quit()

# Export links to txt file
with open('books_urls.txt', 'w') as f:
    for item in final_urls:
        # Avoid exporting the books added in the comments by the people (only two cases)
        if (item != 'https://www.goodreads.com/book/show/2510.The_War_of_the_Worlds' and item != 'https://www.goodreads.com/book/show/1812457.The_Shack'):
            f.write("%s\n" % item)

del urls, final_urls

print("--- %s seconds ---" % (time.time() - start_time))

[WDM] - Driver [C:\Users\Daniel_Jimenez\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


Retrieving page 1 / 300 took 24.5305757522583 seconds
Retrieving page 2 / 300 took 31.544260501861572 seconds
Retrieving page 3 / 300 took 30.777462482452393 seconds
Retrieving page 4 / 300 took 27.54737663269043 seconds
Retrieving page 5 / 300 took 30.077595710754395 seconds
Retrieving page 6 / 300 took 26.381412029266357 seconds
Retrieving page 7 / 300 took 26.7865309715271 seconds
Retrieving page 8 / 300 took 24.992651224136353 seconds
Retrieving page 9 / 300 took 29.19309949874878 seconds
Retrieving page 10 / 300 took 28.588332414627075 seconds
Retrieving page 11 / 300 took 28.364408493041992 seconds
Retrieving page 12 / 300 took 27.344547986984253 seconds
Retrieving page 13 / 300 took 26.75330424308777 seconds
Retrieving page 14 / 300 took 29.226094007492065 seconds
Retrieving page 15 / 300 took 26.680985689163208 seconds
Retrieving page 16 / 300 took 25.645787715911865 seconds
Retrieving page 17 / 300 took 25.48965358734131 seconds
Retrieving page 18 / 300 took 24.455013275146484

As shown, the process took a bit more that 2 hours. Now we are ready to crawl the books.

## 1.2. Crawl books
Now, we need to download in our PC, like HTML, all webpages for each of the books' URL retrieved. To do so, we followed the next way of thinking:

1. First, we need to upload the *books_urls.txt* containing the links to each book
2. Later, we create from Python the 300 folders to store each of the HTML files
3. With the combination of `BeautifulSoup` and `requests` we retrieved the content of each HTML to Python
4. Automatically, we stored each HTML file in the respective folder in our PC, to keep it tidy
5. The points 3 and 4 were executed for each of the 30.000 URLs

Note: To run this process, we proceeded to create 4 different sessions in a single computer and executed the loop by chunks. In each chunk, we had to retrieve 7500 URLs. We used this "parallelization" technique to reduce the execution time.  

In [2]:
# Read the file containing retrieved URLs
with open('books_urls.txt', newline='') as f:
    reader = csv.reader(f)
    final_urls = list(reader)

In [57]:
start_time = time.time()
# Create 300 folders to store the HTML files
for i in range(300):
    # Define path to create folders
    newpath = os.getcwd() + r'\data\HTML_files\page_' + str(i + 1)
    # Check if folder exists, if not, create it
    if not os.path.exists(newpath):
        os.makedirs(newpath)

# Populate each folder with the corresponding HTML books
for i in range(len(final_urls)):
# for i in range(14620,14621):
# for i in [23543,23918,23998,24472,25747,27954]:
    partial_time1 = time.time()
    # Get the link to retrieve
    link = ' '.join(map(str, final_urls[i]))
    
#     print(link)
    # Request the webpage
    page = requests.get(link)
    # Parse page to soup format
    soup = BeautifulSoup(page.text, features='lxml')
    # Calculate the page of the corresponding book
    page = math.ceil((i + 1) / 100)
    
    # Save each webpage as HTML in our PC
    f = open(r"data/HTML_files/page_" + str(page) + r"/book_" + str(i + 1) + r".html", "w", encoding='utf-8')
    f.write(str(soup.prettify()))
    f.close()
    
    print("Processing book", i + 1, "/ 30000 took", time.time() - partial_time1, "seconds")

print("--- %s seconds ---" % (time.time() - start_time))

Processing book 23544 / 30000 took 1.2755396366119385 seconds
Processing book 23919 / 30000 took 0.4957573413848877 seconds
Processing book 23999 / 30000 took 1.281550407409668 seconds
Processing book 24473 / 30000 took 0.9667823314666748 seconds
Processing book 25748 / 30000 took 1.3054816722869873 seconds
Processing book 27955 / 30000 took 1.397230625152588 seconds
--- 6.751603603363037 seconds ---


The previous process consumed a total of 3 hours to download the 30.000 webpages. The next step is to parse all the information of the HTML files downloaded.

## 1.3 Parse downloaded pages


At this point, we are challenged to execute the parsing. It means, extract relevant information regarding the books that are stored in the HTML files. 

First, we defined a function that let us find and retrieve some interesting pieces of information regarding the books (12 attributes). The `scrap_book` function is a modification of the one that we created in LAB class. The latter is leveraged in the use of `find_all` functionality provided by `BeautifullSoup`. Besides, we included some `Try/Except` steps to handle the possible errors when any of the desired attributes of the book don't exist. Furthermore, we used the `langdetect` library to find out the language of each book's description. The definition of this function can be found in the file *our_functions.py*

Later, we continued with the next phases:
    
1. We used the function mentioned above to get the attributes for each book
2. Then, we appended in a .tsv file the result of the retrieved information of every book, separated by \t (TAB)
3. The prebious phases where executed for the whole 30.000 books

Note: Again, we proceeded to initiate 4 sessions in a single computer and executed the loop by chunks. In each chunk, we had to look for the information of 7500 books each. As a result, we will have 4 .tsv files that need to be appended later.

In [3]:
start_time = time.time()

for i in range(0, 7500):
# for i in range(14,16):
    partial_time1 = time.time()

    # Calculate the page of the corresponding book
    page = math.ceil((i + 1) / 100)
    
    # Get the HTML of each book
    path = r"data/HTML_files/page_" + str(page) + r"/book_" + str(i + 1) + r".html"
    
    # Scrap the content
    info_book = our_functions.scrap_book(path)
    
    # Add the URL
    info_book = info_book + final_urls[i]
#     print(info_book)
    info_book =  '\t'.join(map(str,info_book)) + " \n"
    
    # Write in the txv file
    with open("data/tsv _files/final_df_1.tsv", "a", encoding='utf-8') as record_file:
        record_file.write(info_book)
    
    print("Processing book", i + 1, "/ 30000 took", time.time() - partial_time1, "seconds")

print("--- %s seconds ---" % (time.time() - start_time))

Processing book 1 / 30000 took 1.7022335529327393 seconds
Processing book 2 / 30000 took 1.4291777610778809 seconds
Processing book 3 / 30000 took 0.8537194728851318 seconds
Processing book 4 / 30000 took 1.0043158531188965 seconds
Processing book 5 / 30000 took 1.0651531219482422 seconds
Processing book 6 / 30000 took 1.2147505283355713 seconds
Processing book 7 / 30000 took 0.7482569217681885 seconds
Processing book 8 / 30000 took 1.0801126956939697 seconds
Processing book 9 / 30000 took 0.5096359252929688 seconds
Processing book 10 / 30000 took 0.5934140682220459 seconds
Processing book 11 / 30000 took 0.873664379119873 seconds
Processing book 12 / 30000 took 0.6901562213897705 seconds
Processing book 13 / 30000 took 0.5106661319732666 seconds
Processing book 14 / 30000 took 0.885603666305542 seconds
Processing book 15 / 30000 took 0.710101842880249 seconds
Processing book 16 / 30000 took 0.6263227462768555 seconds
Processing book 17 / 30000 took 0.598400354385376 seconds
Processing

The preceding code took 3 hours to run for the 30.000 books. Now it is time to append all the information in one single table.

As mentioned, to create a single data frame with the information parsed from the 30.000 books, we uploaded each .tsv file and appended them in a pandas object. Later, we deleted duplicated caused for multiple runs over the same book. Then we keep only the books with valid information since some links didn't have the correct structure to be scraped. Afterwards, we validated if the plot of the books was in English. To do so, we used the `langdetect` to get the descriptions matching with *en* (English). As a final step, we saved the file in a .tsv file that contains the whole books.

In [7]:
start_time = time.time()
# Upload both parts of the books' information providing the corresponding name of each column
colnames = ['book_title', 'bookSeries', 'bookAuthors', 'ratingValue', 'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'PublishingDate', 'Characters', 'Setting','language','Url']
data = pd.read_csv("data/tsv _files/final_df_1.tsv", sep="\t", names=colnames, header=None, encoding='utf-8')
data_part2 = pd.read_csv("data/tsv _files/final_df_2.tsv", sep="\t", names=colnames, header=None, encoding='utf-8')
data_part3 = pd.read_csv("data/tsv _files/final_df_3.tsv", sep="\t", names=colnames, header=None, encoding='utf-8')
data_part4 = pd.read_csv("data/tsv _files/final_df_4.tsv", sep="\t", names=colnames, header=None, encoding='utf-8')

# Append all dataframes in a single one
data = data.append(data_part2)
data = data.append(data_part3)
data = data.append(data_part4)
del data_part2, data_part3, data_part4

# Delete duplicates caused by multiple runs over the same book
data = data.drop_duplicates()

# Keep the books that have valid information
data = data[data['book_title'].notnull()]

# Check that all the books has an URL
data = data[data['Url'].notnull()]

# Keep only the books which plot is in english
data = data[data['language'] == 'en']

# Export final table to be used in the next points
data.to_csv("data/final_file/all_books_information.tsv", sep = "\t", index=False)
print(data.shape)
print("--- %s seconds ---" % (time.time() - start_time))

(26568, 13)
--- 1.931851863861084 seconds ---


Finally, we scraped the information about 26568 books (removing those without information and which plot is not in English). Now we are ready to proceed with the creation of our search engines.

# 2. Search Engine

More libraries are required for this part.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Import required libraries
import time
import csv
from bs4 import BeautifulSoup
import requests
import pathlib
import math
import os
import pandas as pd
import numpy as np
import re

## Text pre-processing

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, we must pre-process all the information collected for each book by
1. Removing stopwords
2. Removing punctuation
3. Stemming
4. More ?


In [ ]:
books_info = pd.read_csv("all_books_information.tsv", sep="\t", header=0, encoding='utf8', nrows=1000)

stop_words = set(stopwords.words('english'))
stemming = PorterStemmer()
books_info.head(5)

,book_title,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberofPages,PublishingDate,Characters,Setting,language,Url
0,The Hunger Games,(The Hunger Games #1),Suzanne Collins,4.33,"6,408,019","172,545",In the ruins of a place once known as North Am...,374.0,September 14th 2008,"Katniss Everdeen, Peeta Mellark, Cato (Hunger ...","District 12, Panem - Capitol, Panem - Panem",en,https://www.goodreads.com/book/show/2767052-th...
1,Harry Potter and the Order of the Phoenix,(Harry Potter #5),J.K. Rowling,4.50,"2,524,357","42,719",There is a door at the end of a silent corrido...,870.0,September 2004,"Sirius Black, Draco Malfoy, Ron Weasley, Petun...",Hogwarts School of Witchcraft and Wizardry - L...,en,https://www.goodreads.com/book/show/2.Harry_Po...
2,To Kill a Mockingbird,(To Kill a Mockingbird),Harper Lee,4.28,"4,526,703","91,786",The unforgettable novel of a childhood in a sl...,324.0,May 23rd 2006,"Scout Finch, Atticus Finch, Jem Finch, Arthur ...","Maycomb, Alabama",en,https://www.goodreads.com/book/show/2657.To_Ki...
3,Pride and Prejudice,NaN,Jane Austen,4.26,"3,016,917","67,784","Since its immediate success in 1813, ...",279.0,October 10th 2000,"Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabet...","United Kingdom - Derbyshire, England - England...",en,https://www.goodreads.com/book/show/1885.Pride...
4,Twilight,(The Twilight Saga #1),Stephenie Meyer,3.60,"4,988,910","104,902",About three things I was absolutely positive. ...,501.0,September 6th 2006,"Edward Cullen, Jacob Black, Laurent, Renee, Be...","Forks, Washington - Phoenix, Arizona - Washing...",en,https://www.goodreads.com/book/show/41865.Twil...


In [ ]:
def tokenize_row(row):
    tokens = nltk.word_tokenize(row)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

def stem_row(row):
    stemmed_list = [stemming.stem(word) for word in row]
    return (stemmed_list)

def rm_stopwords_row(row):
    meaningful_words = [w for w in row if not w in stop_words]
    return (meaningful_words)


books_info['Plot'] = books_info['Plot'].apply(tokenize_row)

print(books_info['Plot'][0])

books_info['Plot'] = books_info['Plot'].apply(stem_row)

print(books_info['Plot'][0])

books_info['Plot'] = books_info['Plot'].apply(rm_stopwords_row)

print(books_info['Plot'][0])


['In', 'the', 'ruins', 'of', 'a', 'place', 'once', 'known', 'as', 'North', 'America', 'lies', 'the', 'nation', 'of', 'Panem', 'a', 'shining', 'Capitol', 'surrounded', 'by', 'twelve', 'outlying', 'districts', 'The', 'Capitol', 'is', 'harsh', 'and', 'cruel', 'and', 'keeps', 'the', 'districts', 'in', 'line', 'by', 'forcing', 'them', 'all', 'to', 'send', 'one', 'boy', 'and', 'one', 'girl', 'between', 'the', 'ages']
['In', 'the', 'ruin', 'of', 'a', 'place', 'onc', 'known', 'as', 'north', 'america', 'lie', 'the', 'nation', 'of', 'panem', 'a', 'shine', 'capitol', 'surround', 'by', 'twelv', 'outli', 'district', 'the', 'capitol', 'is', 'harsh', 'and', 'cruel', 'and', 'keep', 'the', 'district', 'in', 'line', 'by', 'forc', 'them', 'all', 'to', 'send', 'one', 'boy', 'and', 'one', 'girl', 'between', 'the', 'age']
['In', 'ruin', 'place', 'onc', 'known', 'north', 'america', 'lie', 'nation', 'panem', 'shine', 'capitol', 'surround', 'twelv', 'outli', 'district', 'capitol', 'harsh', 'cruel', 'keep', 'di

## 2.1. Conjunctive query


### 2.1.1 Creating the index


First we compute the list called vocabulary containing all the possible words in our corpus.

In [ ]:
vocabulary = []
for plot in books_info['Plot']:
  for word in plot :
    if word not in vocabulary:
      vocabulary.append(word)

We then create the Inverted index by zipping matching the vocabulary list with the document_ids containing the word. There are no duplicates in our dict (ie. if a word appears two time in a plot it makes no difference it this first index)

In [ ]:
dict_values=[[]for _ in range(len(vocabulary))] #for each word of the vocabulary we'll save which plot contain it (document_id)

document_id=0
for plot in books_info['Plot']:
  for w0rd in plot :
    index=vocabulary.index(w0rd)
    if document_id not in dict_values[index] :
     dict_values[index].append(document_id)
  document_id+=1

Inverted_index=dict(zip(vocabulary, dict_values))

### 2.1.2 Executing the query


We ask the user to input a query. Note that the query words need to go through the stemming process too otherwise we will not be able to match them properly.

In [ ]:
query = input("What kind of book are you looking for ?\n")
query=nltk.word_tokenize(query)
query = [stemming.stem(word) for word in query]
print(query)

What kind of book are you looking for ?
survival game
['surviv', 'game']


In [ ]:
results=[]
for word in query:
  if word in Inverted_index.keys():
    for document_id in Inverted_index[word] :
      results.append(books_info[['book_title','Plot','Url']].iloc[document_id])

result_df= pd.concat(results, axis=1)
result_df=result_df.T #we transpose the dataframe to plot it as specified in the hw description
result_df.head(20)

,book_title,Plot,Url
52,Life of Pi,"[life, Pi, fantasi, adventur, novel, yann, mar...",https://www.goodreads.com/book/show/4214.Life_...
66,Watership Down,"[set, england, onc, idyl, rural, landscap, thi...",https://www.goodreads.com/book/show/76620.Wate...
68,Little Women,"[gener, reader, young, old, male, femal, falle...",https://www.goodreads.com/book/show/1934.Littl...
84,Angela's Ashes,"[imbu, everi, page, frank, mccourt, astound, h...",https://www.goodreads.com/book/show/252577.Ang...
175,The Titan's Curse,"[It, everyday, find, combat, son, greek, god, ...",https://www.goodreads.com/book/show/561456.The...
179,The Call of the Wild,"[first, publish, regard, jack, london, masterp...",https://www.goodreads.com/book/show/1852.The_C...
219,Catching Fire,"[odd, katniss, everdeen, ha, surviv, hunger, g...",https://www.goodreads.com/book/show/6148028-ca...
235,Man's Search for Meaning,"[psychiatrist, viktor, frankl, memoir, ha, riv...",https://www.goodreads.com/book/show/4069.Man_s...
315,A Court of Mist and Fury,"[feyr, surviv, amarantha, clutch, return, spri...",https://www.goodreads.com/book/show/17927395-a...
318,Mockingjay,"[final, book, hunger, game, trilog, thi, new, ...",https://www.goodreads.com/book/show/7260188-mo...


## 2.2  Conjunctive query & Ranking score


## 2.2.1 Inverted index and TF-IDF score

Our goal is to get a matching score for each book and to plot the top-k article. To compute this matching score, we use the tfidf score and the cosine similarity.

Check this document for more info on matching score computation (material recommanded by the professor) : https://nlp.stanford.edu/IR-book/pdf/06vect.pdf

In [ ]:
document_nb=books_info.shape[0] #this is necessary to compute the idf

new_dict_values=[[]for _ in range(len(vocabulary))]
document_id=0
for plot in books_info['Plot']:
  for w0rd in plot :
    index=vocabulary.index(w0rd)
    new_dict_values[index].append(document_id)
  document_id+=1

list=[]
for i in range(len(new_dict_values)):
  list.append(Counter(new_dict_values[i]))

word_occurences=dict(zip(vocabulary,list))

In [ ]:
document_id=0
tfidf_values=[[]for _ in range(len(vocabulary))]
info=[]
words_already_seen=[]
for plot in books_info['Plot']:
  words_nb=len(plot)
  for w0rd in plot :
    if w0rd not in words_already_seen:
      words_already_seen.append(w0rd)
      term_id=vocabulary.index(w0rd)
      tf=word_occurences[w0rd][document_id]/words_nb
      idf=np.log(document_nb/len(word_occurences[w0rd]))
      tfidf=tf*idf
      info.append('document_'+str(document_id))
      info.append(tfidf)
      tfidf_values[term_id].append(info)
      info=[]
  words_already_seen=[]
  document_id+=1

Inverted_index_tfidf=dict(zip(vocabulary,tfidf_values))

print(Inverted_index_tfidf)

{'In': [['document_0', 0.06682558549676333], ['document_8', 0.05383172165017046], ['document_16', 0.05699829351194519], ['document_19', 0.05099847314226675], ['document_20', 0.06251425740019795], ['document_22', 0.10475362050843982], ['document_49', 0.06682558549676333], ['document_50', 0.15299541942680026], ['document_67', 0.1019969462845335], ['document_72', 0.05536977084017533], ['document_81', 0.05383172165017046], ['document_86', 0.04969081998477273], ['document_89', 0.05699829351194519], ['document_91', 0.05536977084017533], ['document_99', 0.08425834693070158], ['document_100', 0.05237681025421991], ['document_103', 0.05872551452745869], ['document_114', 0.04844854948515342], ['document_122', 0.05383172165017046], ['document_135', 0.05699829351194519], ['document_145', 0.0745362299771591], ['document_146', 0.05536977084017533], ['document_148', 0.05536977084017533], ['document_162', 0.09453375509298227], ['document_165', 0.05237681025421991], ['document_170', 0.04726687754649113

# 3. Define a new score!

# 4. Make a nice visualization!

# 5. Algorithmic Question

## 1. Write a recursive program that, given a string, computes the length of the subsequence of maximum length that is in alphabetical order. 
Try some examples. Are the examples of short strings correct? Can you find examples that your algorithm does not terminate in reasonable time?

To calculate the maximum length of a subsequence of characters that is in alphabetical order we created three functions as follows:

`alphabet_pos`: Receives a string and changes the characters by positions of the letters in the alphabet (mapping).

`max_len_sub_alpha_recursive`: Receives a position ($i$) of the character where the string is included, and a list representing the numbers mapped from letters. This is a recursive function that evaluates the maximum length of a subsequence of characters that is in alphabetical order that finishes in the letter of the $i^{th}$ position of the string.

`review_string`: Receives a string and executes the `max_len_sub_alpha` function for each character of the string ($n$ times).

All the previous functions can be found in the file our_functions.py.

In [36]:
start_time = time.time()
# Set the desired string
string = "ABCDE"
# Apply function to string
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.review_string(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 5
--- 0.0 seconds ---


In [37]:
start_time = time.time()
# Set the desired string
string = "KBEAXCZ"
# Apply function to string
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.review_string(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 4
--- 0.0 seconds ---


Looking at the two short examples, we can see that the results are correct since considering the string ABCDE the maximum length of a subsequence of characters that is in alphabetical order is 5. Also, regarding the string K**BE**A**X**C**Z**, the maximum length of a subsequence of characters that is in alphabetical order is 4.

In [18]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSL"
# Apply function to string
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.review_string(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 6
--- 0.0019936561584472656 seconds ---


In [19]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSLKJHDSAKJDSAADSHJHGDSAJHGDSAJDSAJH"
# Apply function to string
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.review_string(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 6
--- 0.023902177810668945 seconds ---


In [20]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSLKJHDSAKJDSAADSHJHGDSAJHGDSAJDSAJHGDSAJHGJHDSKJHLSAKJHDSKJHDSAKJHSAKJWWIUYSKAKJHDLKJHSAB"
# Apply function to string
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.review_string(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 10
--- 3.685804605484009 seconds ---


Now, we considered 3 more examples, where we can see how the longer the string, the larger the execution time is. It makes us believe that the complexity of the recursive program could be exponential.

## 2. Show that the running time of the algorithm is exponential.

Let's consider $T(N)$ as the running time of the algorithm for the given length of the string ($N$). Taking into account that the function `max_len_sub_alpha_recursive` evaluates the maximum length of a subsequence of characters that is in alphabetical order that finishes in the letter of the  $i^{th}$ position of the string, it can be shown that:


$T(N) = T(N-1) + T(N-2) + T(N-3) + T(N-4) + ... + T(1) + T(0)$ 

Now, replacing $T(N-1)$ in the previous formula, we have that:

$T(N) = [T(N-2) + T(N-3) + T(N-4) + ... + T(1) + T(0)] + T(N-2) + T(N-3) + T(N-4) + ... + T(1) + T(0)$ 

$T(N) = 2[T(N-2) + T(N-3) + T(N-4) + ... + T(1) + T(0)]$

Again, replacing $T(N-1)$ in the previous formula, we have that:

$T(N) = 4[T(N-3) + T(N-4) + ... + T(1) + T(0)]$

If we continue to do the replacements in the previous formula, we get that:

$T(N) = 8[T(N-4) + ... + T(1) + T(0)]$

$T(N) = 16[T(N-5) + ... + T(1) + T(0)]$

$T(N) = 2^î[T(N-(i-1)]$

Finally, when taking the value of N we get:

$T(N) = 2^N[T(N-(N-1)]$

$T(N) = 2^N[T(1)]$

$T(N) = 2^N$

In the end, we obtained that the complexity of the algorithm is $O(2^n)$

But, it is important to take into account that we executed the previous algorithm $n$ times, that corresponds to each letter of the substring. The latter means that the final complexity of the algorithm is $O(n2^n)$. Effectively, the running time of the algorithm is exponential.

## 3. Write a program that computes the length of the subsequence of maximum length, using dynamic programming.

To calculate the maximum length of a subsequence of characters that is in alphabetical order, but using dynamic programming, we created one functions as follows:

`max_len_sub_alpha_dyn_prog`: Receives a string and maps its letters to numbers (using the function `alphabet_pos`). Later we defined an auxiliary array to store the maximum length that is finding when visiting the positions of the arrays with two for loops to evaluate the conditions defined for a subsequence to be in alphabetical order. 

The previous function can be found in the file our_functions.py.

In [47]:
start_time = time.time()
# Set the desired string
string = "ABCDE"
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.max_len_sub_alpha_dyn_prog(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 5
--- 0.0009949207305908203 seconds ---


In [46]:
start_time = time.time()
# Set the desired string
string = "KBEAXCZ"
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.max_len_sub_alpha_dyn_prog(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 4
--- 0.0009925365447998047 seconds ---


Looking at the two short examples, we can see that the results are correct since considering the string ABCDE the maximum length of a subsequence of characters that is in alphabetical order is 5. Besides, regarding the string K**BE**A**X**C**Z**, the maximum length of a subsequence of characters that is in alphabetical order is 4. The results are consistent with those obtained with the recursive algorithm.

In [27]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSL"
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.max_len_sub_alpha_dyn_prog(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 6
--- 0.0009989738464355469 seconds ---


In [23]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSLKJHDSAKJDSAADSHJHGDSAJHGDSAJDSAJH"
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.max_len_sub_alpha_dyn_prog(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 6
--- 0.0009565353393554688 seconds ---


In [25]:
start_time = time.time()
# Set the desired string
string = "KJHDSALKJHALJHFDKJDLKHSLKJHDSAKJDSAADSHJHGDSAJHGDSAJDSAJHGDSAJHGJHDSKJHLSAKJHDSKJHDSAKJHSAKJWWIUYSKAKJHDLKJHSAB"
print("The maximum length of a subsequence of characters that is in alfabetical order is", our_functions.max_len_sub_alpha_dyn_prog(string))
print("--- %s seconds ---" % (time.time() - start_time))

The maximum length of a subsequence of characters that is in alfabetical order is 10
--- 0.0009975433349609375 seconds ---


Finally, considering the same examples that in the previous exercise of the application of the recursive program, the execution time remains almost the same when increasing the number of letters considerably in the string. It shows that the dynamic program is way better than the recursive one since the complexity of the former is of polynomial order.

## 4. What is its runtime complexity?

With the previous algorithm, we can see that: 

- The first for loop goes from the second letter to the $n^{th}$ letter ($n$ is the length of the string). The previous means that it visits $n - 1$ letters. 
- The second for loop (that is contained in the first loop) goes from the first letter to the $i^{th}$. That means that, when $i = n$, it will go from the first letter to the $(n-1)^{th}$ letter.

Then, the running time can be seen as:

$T(N) <= (N-1) * (N-1)$

$T(N) <= (N^2 - 2N + 1)$

And taking the worst case, we have that the running time is:
$O(n^2)$

# BONUS

To prove that given

$X[i] =$ "the length of the longest sequence of characters in alphabetical order that terminates at the $i^{th}$ character".

$X[i] = 1 + max(X[j]; j = 0, ..., i-1 ,$ such that $S[j]<S[i])$

$X[i] = 1,$ if there does not exist such a j

We will use the induction approach as follows:

- First, we prove the base case. It is that $X[1] = 1$. This one is correct because it is taking the first element as a subsequence of itself of length 1.

- Now we use an inductive case. For that, we can assume that $X[1], ..., X[i-1]$ are correct.
- Then, at index $i$ (the next index) the first possible subsequence is the element $S[i]$ itself of length 1.
- Another possibility is a subsequence starting before and finishing at $i$, with the last index included being $j$.
- The latter only happens when $S[j] < S[i]$, because the order needs to be alphabetical.
- If the previous occurs, then its maximum length is $X[j] + 1$ since we appended the letter at index $i$. 
- So $X[i]$ is correct because the algorithm calculates the maximum of all these possibilities.